# Data Science Case Study

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

from shapely.geometry import Point
import geopandas as gpd

import holidays

plt.style.use('ggplot')
pd.options.display.max_columns = 500
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Read Data

In [ ]:
data_path = Path.cwd().resolve().parents[1] / "data"
sales = pd.read_csv(data_path / "sales.csv", index_col=0)
stations = pd.read_csv(data_path / "stations.csv", index_col=0)

___Sales Data Dictionary___

    - date: yyyy-mm-dd format. increments of one day. not unique.
    - sales: number of sales on that given day, relative to the station
    - station: name of a station in the UK

___Stations Data Dictionary___

    - station: name of the station in the UK
    - lat: latitude coordinate
    - lon: longitude coordinate
    - operator: responible entity for the ticket / journey

## Operator Observation

    Trainline is a different kind of entity to English / Welsh / Scottish Rail. 
    The latter is a private or public body that own the trains and journeys
    the other is a vendor which sells tickets on behalf of these bodies.

***Why this matters***

    Railways may sell tickets through many channels and Trainline is simply just one of them (i.e. tickets at station, their own website etc). A sale to trainline is also a sale to whichever railway operates at the station.

    We will need to map stations to the correct body based on the lat/lon data so that sales totals for each railway is fully represenative. 
    (e.g. when lat/lon falls inside the Wales then attribute sales to Welsh Railway.)

***Risks***

    Trainline take a portion of the ticket sales as revenue and the rest is passed onto the body that owns the station. Care will need to be taken to ensure reasonable estimates.


***From now on***

***Railways - refers to the english/welsh/scottish railway companies***

***Trainline - will be refered to by its namesake***


## Engineering Features & Cleaning

### Mapping stations to nations

Downloaded GeoJSON object from Gov Website for mapping
https://geoportal.statistics.gov.uk/datasets/e4cab2a2419f46d7847dafb50a159aa7_0/explore

In [ ]:
# From my understanding given the resolution of the gov file, error could be up to 1000m. 
# This is forgivable given that nations are large. 
# If error is over 1000m then it is likely mapped poorly

# Convert to GeoDataFrame
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["lon"], stations["lat"]),
    crs="EPSG:4326"
)
countries = gpd.read_file(data_path / "countries.geojson").to_crs(gdf_stations.crs)

# Buffer stations by x meters then check point-in-polygon
gdf_stations_m = gdf_stations.to_crs(27700)
countries_m = countries.to_crs(27700)
x = 1000
gdf_stations_buf = gdf_stations_m.copy()
gdf_stations_buf["geometry"] = gdf_stations_buf.buffer(x)

joined = gpd.sjoin(
    gdf_stations_buf,
    countries_m[["CTRY22NM", "geometry"]],
    how="left",
    predicate="intersects"
)
gdf_stations_w_nation = (
    joined.rename(columns={"CTRY22NM": "nation"})
          .drop(columns=["geometry", "index_right"], errors="ignore")
)

# Checking errors clear
gdf_stations_w_nation.query("nation.isna()")

In [ ]:
# Sanity Check our coordinates - none should have errors above 1000m
station = "Westcliff"
nation = "England"

def _pick_station_row(df: gpd.GeoDataFrame, station: str, prefer_non_null_nation=True):
    sub = df.loc[df["station"] == station]
    if sub.empty:
        raise ValueError(f"Station '{station}' not found.")
    if prefer_non_null_nation and "nation" in sub.columns and sub["nation"].notna().any():
        sub = sub[sub["nation"].notna()]
    return sub.iloc[0]

def zoom_station(df: gpd.GeoDataFrame, station: str, pad_deg: float = 1.0):
    row = _pick_station_row(df, station)
    pt = gpd.GeoSeries([Point(row.lon, row.lat)], crs="EPSG:4326")

    ax = countries.plot(figsize=(6, 6), edgecolor="purple", facecolor="none")
    pt.plot(ax=ax, markersize=80)
    plt.xlim(row.lon - pad_deg, row.lon + pad_deg)
    plt.ylim(row.lat - pad_deg, row.lat + pad_deg)
    title_nation = row["nation"] if "nation" in row else None
    plt.title(f"{station} (nation={title_nation})")
    plt.show()

def location_offset_error(station: str, nation: str) -> float:
    row = _pick_station_row(gdf_stations, station, prefer_non_null_nation=False)
    pt = gpd.GeoSeries([Point(row.lon, row.lat)], crs="EPSG:4326").to_crs(27700).iloc[0]
    sel = countries.loc[countries["CTRY22NM"] == nation]
    if sel.empty:
        raise ValueError(f"Nation '{nation}' not found in countries['CTRY22NM'].")
    poly = sel.to_crs(27700).geometry.union_all()
    dist_m = pt.distance(poly)
    print(f"{station} → {nation}: {dist_m:.1f} m")
    return float(dist_m)

zoom_station(df=gdf_stations_w_nation,station=station)
location_offset_error(station=station, nation=nation)


In [ ]:
# Now given the nation data we can map stations to railways
gdf_stations_w_nation["railway"] = gdf_stations_w_nation.apply(
    lambda row: (
        "English Rail" if row["operator"] == "Trainline" and row["nation"] == "England" else
        "Scottish Rail" if row["operator"] == "Trainline" and row["nation"] == "Scotland" else
        "Welsh Rail" if row["operator"] == "Trainline" and row["nation"] == "Wales" else
        row["operator"]
    ),
    axis=1
)
# Add a flag for trainline figures
gdf_stations_w_nation["trainline_assisted_sales"] = (gdf_stations_w_nation["operator"] == "Trainline").astype(bool)
gdf_stations_w_nation.rename(columns={"operator": "source_operator"}, inplace=True)


In [ ]:
# Check one-to-one realationship between stations and operators before engineering steps
gdf_stations_w_nation.groupby('station')['source_operator'].nunique().sort_values(ascending=False)

In [ ]:
gdf_stations_w_nation.query("station == 'Exeter Central'")

In [ ]:
# Bottom one is correct
zoom_station(
    df=gdf_stations_w_nation.query("station == 'Exeter Central' & source_operator == 'Trainline'"),
    station='Exeter Central'
), zoom_station(
    df=gdf_stations_w_nation.query("station == 'Exeter Central' & source_operator == 'English Rail'"),
    station='Exeter Central'
)

In [ ]:
# Because of the incorrect lat/lon we are dropping the row. 
# Hard to say whether Exeter Central should update to be Trainline or whether the Trainline entry is wrong
# As the row was wrong we shall leave Exeter as False for Trainline Assisted Sales
gdf_stations_w_nation = gdf_stations_w_nation[~((gdf_stations_w_nation.station == "Exeter Central") & (gdf_stations_w_nation.source_operator == "Trainline"))]

### Join datasets

    It looks like the stations don't match 1-1 between the datasets.

    We will need to use the coordinates given to match to downloaded polygons attributing to the area names given to us i.e. Gloucester

In [ ]:
sales.head()

In [ ]:
local_authority = gpd.read_file(data_path / "local_authority.geojson").to_crs(sales.crs)

# Buffer stations by x meters then check point-in-polygon
gdf_stations_m = gdf_stations.to_crs(27700)
countries_m = countries.to_crs(27700)
x = 1000
gdf_stations_buf = gdf_stations_m.copy()
gdf_stations_buf["geometry"] = gdf_stations_buf.buffer(x)

joined = gpd.sjoin(
    gdf_stations_buf,
    countries_m[["CTRY22NM", "geometry"]],
    how="left",
    predicate="intersects"
)
gdf_stations_w_nation = (
    joined.rename(columns={"CTRY22NM": "nation"})
          .drop(columns=["geometry", "index_right"], errors="ignore")
)

# Checking errors clear
gdf_stations_w_nation.query("nation.isna()")

In [ ]:
gdf_stations_w_nation.query("trainline_assisted_sales == True")

In [ ]:
sales.station.unique()


In [ ]:
gdf_stations_w_nation.station.unique()

In [ ]:
station_sales = pd.merge(sales, gdf_stations_w_nation[["station", "nation", "railway", "trainline_assisted_sales"]], on="station", how="left")

### Adding Date time Features

In [ ]:
station_sales["date"] = pd.to_datetime(station_sales["date"], format="%Y-%m-%d")

station_sales["year"] = station_sales["date"].dt.year
station_sales["month"] = station_sales["date"].dt.month
station_sales["day"] = station_sales["date"].dt.day
station_sales["day_of_week"] = station_sales["date"].dt.day_name()
station_sales["quarter"] = station_sales["date"].dt.quarter
station_sales["is_weekend"] = station_sales["date"].dt.dayofweek >= 5

uk_holidays = holidays.UnitedKingdom(years=station_sales['date'].dt.year.unique())
station_sales['is_holiday'] = station_sales['date'].isin(uk_holidays)
station_sales['holiday_name'] = station_sales['date'].map(uk_holidays)

### Number of Sales Adjustments

***How do we interpret 'sales'?***

    It could be number of ticket sales or the gross revenue. It is hard to say since the value is a decimal.

    Spot checking I think it makes more sense to count these as ticket sales

***Why?***

    11/02/202 | 92.31415573	| Stratford (London) 
    is an example of a row.

    Stratford is a busy station so the station only making £92.31 is unlikely (thats like a few tickets)
    Whereas it being the number of tickets is a little more believeable, although trains can have thousands of footfall in one day. I assume for this case study data is at least somewhat synthetic at minimum.

***Actions***

    You can't have a decimal number of tickets in one day. We will round to the nearest whole number


In [ ]:
# Rounding
station_sales['sales'] = station_sales['sales'].round().astype(int)

#### Adding lags and rolling features for sales by groups

In [ ]:
def add_rolling_features(
    df: pd.DataFrame,
    feature: str = "sales",
    group_col: str = "station",
    date_col: str = "date",
    windows: list | None = None,
) -> pd.DataFrame:
    if windows is None:
        windows = [1, 3, 7, 14, 30, 60, 90, 180]

    out = df.copy()
    out.sort_values([group_col, date_col], inplace=True)

    cols = []
    for w in windows:
        cols += [
            f"{group_col}_by_{feature}_mean_{w}d",
            f"{group_col}_by_{feature}_std_{w}d",
            f"{group_col}_by_{feature}_sum_{w}d",
        ]
    for c in cols:
        out[c] = np.nan

    for _, g in out.groupby(group_col, sort=False):
        s = g.set_index(date_col)[feature]

        for w in windows:
            win = f"{w}D"
            mean_vals = s.rolling(win, min_periods=1).mean().to_numpy()
            std_vals  = s.rolling(win, min_periods=1).std().to_numpy()
            sum_vals  = s.rolling(win, min_periods=1).sum().to_numpy()

            out.loc[g.index, f"{group_col}_by_{feature}_mean_{w}d"] = mean_vals
            out.loc[g.index, f"{group_col}_by_{feature}_std_{w}d"]  = std_vals
            out.loc[g.index, f"{group_col}_by_{feature}_sum_{w}d"]  = sum_vals

    return out

def add_lag_features(
    df: pd.DataFrame,
    feature: str = "sales",
    group_col: str = "station",
    date_col: str = "date",
    windows: list | None = None,
    daily_agg: str = "sum",
) -> pd.DataFrame:
    if windows is None:
        windows = [1, 3, 7, 14, 30, 60, 90, 180]

    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors="coerce")
    out.sort_values([group_col, date_col], inplace=True)

    for w in windows:
        out[f"{group_col}_by_{feature}_lag_{w}d"] = np.nan

    # compute per group
    for _, g in out.groupby(group_col, sort=False):
        daily = (
            g[[date_col, feature]]
            .groupby(date_col, as_index=True)
            .agg({feature: daily_agg})
            .sort_index()
        )

        for w in windows:
            lag_daily = daily[feature].shift(freq=f"{w}D")
            vals = lag_daily.reindex(g[date_col]).to_numpy()
            out.loc[g.index, f"{group_col}_by_{feature}_lag_{w}d"] = vals

    return out

In [ ]:
station_sales = (
    station_sales
      .pipe(add_rolling_features, group_col="station", feature="sales")
      .pipe(add_rolling_features, group_col="railway", feature="sales")
      .pipe(add_rolling_features, group_col="nation",  feature="sales")
      .pipe(add_lag_features, group_col="station", feature="sales")
      .pipe(add_lag_features, group_col="railway", feature="sales")
      .pipe(add_lag_features, group_col="nation",  feature="sales")
)

In [ ]:
station_sales.head(1)

### Trainline Features



In [ ]:
df = station_sales.copy()

# 1) Normalize to strict bool, treating NaN as False
df['tl_flag'] = (
    df['trainline_assisted_sales']
      .fillna(False)
      .astype(bool)
)

# 2) Enforce station-level consistency (take any True observed for a station)
station_tl = (df.groupby('station', as_index=False)['tl_flag']
                .any()
                .rename(columns={'tl_flag':'station_tl_flag'}))

# df = df.drop(columns='tl_flag').merge(station_tl, on='station', how='left')
# df['station_tl_flag'] = df['station_tl_flag'].fillna(False)

# # 3) Compute TL sales: sales * flag
# df['trainline_sales'] = df['sales'] * df['station_tl_flag'].astype(int)

# # 4) Aggregate per date + nation
# agg_nation = (df.groupby(['date','nation'], as_index=False)
#                 .agg(total_sales=('sales','sum'),
#                      trainline_sales=('trainline_sales','sum')))
# agg_nation['trainline_share'] = agg_nation['trainline_sales'] / agg_nation['total_sales']

# # 5) Aggregate per date + railway (totals are for that day only because 'date' is in the groupby)
# agg_railway = (df.groupby(['date','railway'], as_index=False)
#                 .agg(total_sales=('sales','sum'),
#                      trainline_sales=('trainline_sales','sum')))
# agg_railway['trainline_share'] = agg_railway['trainline_sales'] / agg_railway['total_sales']


In [ ]:
df['trainline_assisted_sales'].value_counts()